In [1]:
%%capture
!pip install polars
!pip install catboost

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, log_loss

### Можно ли решить задачу с наскока?

Сперва стоит подгрузить данные и посмотреть на то, с чем нам придётся работать.

***Признаки и доступные данные для проекта.***

**Данные о взаимодействии Пользователя и рекламной компании (train.parquet и test.parquet)**
-  |-- platform_id: id платформы (Android, Ios и т.п.)
-  |-- user_id: id Пользователя
-  | -- adv_campaign_id: id рекламной компании
-  |-- target: клик / не клик
-  | -- banner_code: код баннера
-  | -- adv_creative_id: индификатор креатива
-  | -- event_date: date Дата показа рекламной кампании пользователю
-  |-- is_main: boolean True - показ рекламы был осуществлен с главной страницы

**Данные о категориях (categories.parquet)**
- |-- microcat_id: id микрокатегории
- |-- level_id: id уровня в дереве микрокатегорий
- |-- parent_microcat_id: id родительской микрокатегории
- |-- logcat_id: id логической категории
- |-- vertical_id: id вертикали
- |-- category_id: id категории 
 

**Данные о рекламной компании (campaigns_meta.parquet)**
- |-- adv_campaign_id: id рекламной компании 
- |-- start_date: date дата начала рекламной компании 
- |-- end_date: date дата завершения рекламной компании
- |-- goal_cost: цена за клик на рекламу
- |-- goal_budget: общий бюджет рекламной компании
- |-- logcat_id: id логической категории товаров из рекламной кампании
- |-- location_ids: id локации, на которую рекламная компания распространяется 


Для базового решения будем использовать только сет train

In [3]:
df_train = pl.read_parquet('https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/AvitoTechMLcup2024/data/train.parquet')

df_test = pl.read_parquet('https://storage.yandexcloud.net/ds-ods/files/files/0d7b7c0f/test.parquet') # датасет для предсказания

In [4]:
df_train = df_train.with_columns([
    pl.col("adv_campaign_id").cast(pl.Int16), 
    pl.col("adv_creative_id").cast(pl.Int16),  
    pl.col("platform_id").cast(pl.Int8),       
    pl.col("banner_code").cast(pl.Int8),       
    pl.col("target").cast(pl.Int8)               
])

df_test = df_test.with_columns([
    pl.col("adv_campaign_id").cast(pl.Int16),
    pl.col("adv_creative_id").cast(pl.Int16),  
    pl.col("platform_id").cast(pl.Int8),        
    pl.col("banner_code").cast(pl.Int8),        
])


In [5]:
print('Данные для обучения:', df_train.shape)
print('Тестовые данные:', df_test.shape)

Данные для обучения: (114741035, 8)
Тестовые данные: (1983287, 7)


In [6]:
df_train.null_count()

user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0


In [7]:
df_test.null_count()

user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0


In [8]:
df_train.dtypes, df_test.dtypes

([Int64, Int16, Int8, Int16, Date, Int8, Boolean, Int8],
 [Int64, Int16, Int8, Int16, Date, Int8, Boolean])

У нас все числовые признаки, кроме event_date(это дата). 

Числовые признаки можно отнести к категориальным.

Вытащим из даты месяц и день недели.

In [9]:
# Извлечение дня недели, месяца
df_train = df_train.with_columns([
    pl.col("event_date").dt.weekday().alias("day_of_week"), 
    pl.col("event_date").dt.month().alias("month"),
    pl.col('event_date').dt.week().alias('week')
])

df_test = df_test.with_columns([
    pl.col("event_date").dt.weekday().alias("day_of_week"), 
    pl.col("event_date").dt.month().alias("month"),
    pl.col('event_date').dt.week().alias('week')
])

df_train = df_train.drop('event_date')
df_test = df_test.drop('event_date')

## Переходим к предсказанию.

Для базового решения будем использовать CatBoost так как он хорошо работает с категориальными признаками.

При разделение данных на трейн и тест важно учитывать, что не должно создаваться ликов(утечки данных).
Поэтому разделение будем производить по номеру недели.

In [10]:
X_train = df_train.filter(pl.col('week') < 38)
X_test = df_train.filter(pl.col('week') == 38) 
y_train = X_train['target'].to_pandas()
y_test = X_test['target'].to_pandas()

X_train = X_train.drop('target').to_pandas()
X_test = X_test.drop('target').to_pandas()
print(X_train.shape, X_test.shape)

(61060488, 9) (53680547, 9)


In [12]:
cat_features = ['platform_id', 'user_id', 'adv_campaign_id', 'banner_code', 'adv_creative_id', 'is_main', 'day_of_week', 'month', 'week']

# Инициализация и обучение модели
model = CatBoostClassifier(iterations=100, verbose=10)
model.fit(X_train, y_train, cat_features=cat_features)

# Предсказание
y_pred = model.predict_proba(X_test)[:, 1]

# Оценка модели
print('ROC-AUC:', roc_auc_score(y_test, y_pred))
print('LogLoss:', log_loss(y_test, y_pred))

Learning rate set to 0.5
0:	learn: 0.3211484	total: 5.58s	remaining: 9m 11s
10:	learn: 0.0319750	total: 30.3s	remaining: 4m 4s
20:	learn: 0.0315663	total: 54.5s	remaining: 3m 24s
30:	learn: 0.0315248	total: 1m 16s	remaining: 2m 50s
40:	learn: 0.0315186	total: 1m 37s	remaining: 2m 20s
50:	learn: 0.0315140	total: 2m	remaining: 1m 55s
60:	learn: 0.0315118	total: 2m 22s	remaining: 1m 30s
70:	learn: 0.0315100	total: 2m 44s	remaining: 1m 7s
80:	learn: 0.0315088	total: 3m 6s	remaining: 43.7s
90:	learn: 0.0315083	total: 3m 28s	remaining: 20.6s
99:	learn: 0.0315071	total: 3m 48s	remaining: 0us
ROC-AUC: 0.5704166327581525
LogLoss: 0.055637654754094344


In [18]:
fin_data = df_test.to_pandas()
# Прогноз вероятности клика на тестовых данных
y_pred_prob = model.predict_proba(fin_data)[:, 0]

# Выводим вероятности
print("Вероятности клика:", y_pred_prob)

Вероятности клика: [0.9890774  0.9890774  0.9890774  ... 0.97972347 0.97291361 0.97972347]


In [19]:
sample_submission = pl.read_csv('https://storage.yandexcloud.net/ds-ods/files/files/d901afa5/sample_submission.csv') 
sample_submission = sample_submission.with_columns(pl.lit(y_pred_prob).alias("predict"))
sample_submission.write_csv('sample_submission.csv')

## Проверка в таблице лидербордов на финальных данных

Получаем результат = 0,5100039010997964, такой результат говорит о том, что наша модель справилась не лучше чем простое угадывание.		